In [3]:
# import os
# os.chdir('../')
%pwd

'/Users/abrar/ML-DS-Projects/Practice/End-to-End-ML-Project-with-MLOps'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

In [11]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH, schema_filepath=SCHEMA_FILE_PATH):   
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            alpha=params.alpha,
            l1_ratio=params.l1_ratio,
            target_column=schema.name
        )

        return model_trainer_config


In [8]:
import os
from src.mlProject import logger
from sklearn.linear_model import ElasticNet
import pandas as pd
import joblib

In [17]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)

        train_x =  train_data.drop(columns=[self.config.target_column], axis=1)
        train_y = train_data[self.config.target_column]

        model = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        model.fit(train_x, train_y)

        joblib.dump(model, os.path.join(self.config.root_dir, self.config.model_name))


In [18]:
try:
    config_mgr = ConfigurationManager()
    model_trainer_config = config_mgr.get_model_trainer_config()
    model_trainer = ModelTrainer(model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2025-01-03 20:41:06,264: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-01-03 20:41:06,268: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-03 20:41:06,315: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-01-03 20:41:06,324: INFO: common: created directory at: artifacts]
[2025-01-03 20:41:06,336: INFO: common: created directory at: artifacts/model_trainer]
